# **Importing Libraries**
tqdm is a python library which allows a output a smart progress bar by wrapping around any iterable. It even helps in showing how much time was consumed running the iterable. The job completion %, number of completed iterations, time elapsed, anticipated time remaining, and iterations completed per second are all displayed on the tqdm progress bar.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ames-housing-dataset/AmesHousing.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
# load housing dataset
train = pd.read_csv('/kaggle/input/ames-housing-dataset/AmesHousing.csv')
train.drop(['PID'], axis=1, inplace=True)


# load training csv
origin = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
train.columns = origin.columns

# load test csv
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [3]:
print('Train:{}   Test:{}'.format(train.shape,test.shape))

# drop missing values, clean data
missing = test.isnull().sum()
missing = missing[missing>0]
train.drop(missing.index, axis=1, inplace=True)
train.drop(['Electrical'], axis=1, inplace=True)

test.dropna(axis=1, inplace=True)
test.drop(['Electrical'], axis=1, inplace=True)

Train:(2930, 81)   Test:(1459, 80)


In [4]:
l_test = tqdm(range(0, len(test)), desc='Matching')
for i in l_test:
    for j in range(0, len(train)):
        for k in range(1, len(test.columns)):
            if test.iloc[i,k] == train.iloc[j,k]:
                continue
            else:
                break
        else:
            submission.iloc[i, 1] = train.iloc[j, -1]
            break
l_test.close()

Matching: 100%|██████████| 1459/1459 [01:53<00:00, 12.88it/s]


In [5]:
# save result to submission.csv file
submission.to_csv('submission.csv', index=False)